# Dynamic Case Preparation

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=30)

In [4]:
opf_cases = ['./ACTIVSg200/IL200_opf_20wt.xlsx',
             './ACTIVSg200/IL200_opf_50wt.xlsx',
             './ACTIVSg200/IL200_opf_80wt.xlsx',
             ]
sp1 = ams.load(opf_cases[0],
               setup=True, no_output=True,
               default_config=True)
sp2 = ams.load(opf_cases[1],
               setup=True, no_output=True,
               default_config=True)
sp3 = ams.load(opf_cases[2],
               setup=True, no_output=True,
               default_config=True)

In [5]:
for ss in [sp1, sp2, sp3]:
    stg_idx = ss.StaticGen.get_all_idxes()
    stg_wt = ss.StaticGen.find_idx(keys='genfuel', values=['wind'], allow_all=True)[0]
    sn_wt = ss.StaticGen.get(src='Sn', attr='v', idx=stg_wt)
    sn_total = ss.StaticGen.get(src='Sn', attr='v', idx=stg_idx)

    print('Wind penetration: {0:.2f}%'.format(
        sn_wt.sum() / sn_total.sum() * 100))

Wind penetration: 19.75%
Wind penetration: 55.96%
Wind penetration: 76.83%


## Inspect Case

There are different types of generators:
- ST: Steam Turbine (includes nuclear, geothermal, and solar steam)
- NB: ST - Boiling Water Nuclear Reactor
- W2: Wind Turbine, Type 2
- GT: Combustion (Gas) Turbine

In [6]:
stg_dfs = [sp1.StaticGen.as_df(), sp2.StaticGen.as_df(), sp3.StaticGen.as_df()]
columns = ['gentype', 'genfuel']

stg_df = pd.DataFrame()
stg_df['idx'] = stg_dfs[0]['idx']
stg_df['Bus'] = stg_dfs[0]['bus']
stg_df['Sn'] = stg_dfs[0]['Sn']

for i, stg in enumerate(stg_dfs):
    for col in columns:
        stg_df[f'{col}_{20 + i * 30}wt'] = stg[col]

stg_df

,idx,Bus,Sn,gentype_20wt,genfuel_20wt,gentype_50wt,genfuel_50wt,gentype_80wt,genfuel_80wt
0,47,189,682.98,NB,nuclear,NB,nuclear,NB,nuclear
1,1,49,5.44,ST,coal,ST,coal,ST,coal
2,2,50,5.44,ST,coal,ST,coal,ST,coal
3,3,51,5.44,ST,coal,ST,coal,ST,coal
4,4,52,5.44,ST,coal,ST,coal,ST,coal
5,5,53,10.88,ST,coal,ST,coal,ST,coal
6,6,65,180.48,W2,wind,W2,wind,W2,wind
7,7,67,5.64,ST,coal,ST,coal,ST,coal
8,8,68,33.50,ST,coal,ST,coal,W2,wind
9,9,69,33.50,ST,coal,ST,coal,W2,wind


## Dynamic Models Replacement

In the original dynamic case, only ``GENROU`` and ``TGOV1`` is used.
To better represent the dynamic behavior, following replacements are made:
1. W2 type gen, replace GENROU with Wind Generations
1. ST type gen, replace its TGOV1N with IEEEG1
1. GT type gen, replace its TGOV1N with GAST

In [7]:
# load the dynamics case but don't set up it
dyn_base_case = './ACTIVSg200/IL200_dyn.xlsx'

sa1 = andes.load(dyn_base_case,
                 setup=False, no_output=True,
                 default_config=True)
sa2 = andes.load(dyn_base_case,
                 setup=False, no_output=True,
                 default_config=True)
sa3 = andes.load(dyn_base_case,
                 setup=False, no_output=True,
                 default_config=True)

Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.
Generating code for 1 models on 12 processes.


In [8]:
for sp, sa in zip([sp1, sp2, sp3], [sa1, sa2, sa3]):
    # In the ANDES case, get the StaticGen idx for type W2, ST, GT, respectively
    stg_idxes = sp.StaticGen.find_idx(keys='gentype',
                                      values=['W2', 'ST', 'GT'],
                                      allow_all=True)
    stg_w2t = stg_idxes[0]
    stg_st = stg_idxes[1]
    stg_gt = stg_idxes[2]

    n_w2t = len(stg_w2t)
    n_st = len(stg_st)
    n_gt = len(stg_gt)

    # In the ANDES case, get the corresponding SynGen idx
    syg_w2t = sa.SynGen.find_idx(keys='gen', values=stg_w2t)
    syg_st = sa.SynGen.find_idx(keys='gen', values=stg_st)
    syg_gt = sa.SynGen.find_idx(keys='gen', values=stg_gt)

    # Turn off the W2TG, as they will be repalced with renewable generators
    # NOTE: we use `alter` here to ensure the changes can be saved
    # in the exported case
    sa.SynGen.alter(src='u', value=0, idx=syg_w2t)

    # In the ANDES case, get the corresponding TurbineGov idx
    tg_w2t = sa.TurbineGov.find_idx(keys='syn', values=syg_w2t)
    tg_st = sa.TurbineGov.find_idx(keys='syn', values=syg_st)
    tg_gt = sa.TurbineGov.find_idx(keys='syn', values=syg_gt)

    n_gt = len(tg_gt)
    n_st = len(tg_st)
    n_w2t = len(tg_w2t)

    # --- convert TGOV1N attached to ST type generators to TGOV1N ---
    IEEEG1 = pd.DataFrame()

    # mapped parameters
    IEEEG1['u'] = sa.TurbineGov.get(src='u', idx=tg_st)
    IEEEG1['idx'] = [f'IEEEG1_{i}' for i in range(1, n_st+1)]
    IEEEG1['syn'] = sa.TurbineGov.get(src='syn', idx=tg_st)
    IEEEG1['syn2'] = [None] * n_st
    IEEEG1['Tn'] = sa.TurbineGov.get(src='Tn', idx=tg_st)
    IEEEG1['wref0'] = sa.TGOV1N.get(src='wref0', idx=tg_st)
    IEEEG1['K'] = [1/R for R in sa.TurbineGov.get(src='R', idx=tg_st)]
    IEEEG1['T1'] = sa.TurbineGov.get(src='T1', idx=tg_st)
    IEEEG1['PMAX'] = sa.TurbineGov.get(src='VMAX', idx=tg_st)
    IEEEG1['PMIN'] = sa.TurbineGov.get(src='VMIN', idx=tg_st)
    IEEEG1['T2'] = sa.TurbineGov.get(src='T2', idx=tg_st)
    IEEEG1['T5'] = sa.TurbineGov.get(src='T3', idx=tg_st)

    # unmapped parameters are skipped and default values are used
    # UO, UC, T3, T4, T6, T7, K1, K3, K5, K7, K2, K4, K6, K8

    # add the IEEEG1 to the system
    for row in IEEEG1.itertuples(index=False):
        sa.add(model='IEEEG1', param_dict={**row._asdict()})

    # turn off the original governors
    sa.TurbineGov.alter(src='u', value=0, idx=tg_st)

    # --- convert TGOV1N attached to GT type generators to GAST ---
    GAST = pd.DataFrame()
    # mapped parameters
    GAST['u'] = sa.TurbineGov.get(src='u', idx=tg_gt)
    GAST['idx'] = [f'GAST_{i}' for i in range(1, n_gt+1)]
    GAST['syn'] = sa.TurbineGov.get(src='syn', idx=tg_gt)
    GAST['Tn'] = sa.TurbineGov.get(src='Tn', idx=tg_gt)
    GAST['wref0'] = sa.TurbineGov.get(src='wref0', idx=tg_gt)
    GAST['R'] = sa.TurbineGov.get(src='R', idx=tg_gt)
    GAST['VMAX'] = sa.TurbineGov.get(src='VMAX', idx=tg_gt)
    GAST['VMIN'] = sa.TurbineGov.get(src='VMIN', idx=tg_gt)
    GAST['T2'] = sa.TurbineGov.get(src='T2', idx=tg_gt)
    GAST['T3'] = sa.TurbineGov.get(src='T3', idx=tg_gt)
    GAST['Dt'] = sa.TurbineGov.get(src='Dt', idx=tg_gt)

    # unmapped parameters are skipped and default values are used
    # KT, AT

    # add the GAST to the system
    for row in GAST.itertuples(index=False):
        sa.add(model='GAST', param_dict={**row._asdict()})

    # turn off the original governors
    sa.TurbineGov.alter(src='u', value=0, idx=tg_gt)

    # --- GENROU -> WT2G: REGCA1 ---
    REGCA1 = pd.DataFrame()

    # mapped parameters
    REGCA1['u'] = sa.SynGen.get(src='u', idx=syg_w2t)
    REGCA1['idx'] = [f'REGCA_{i}' for i in range(1, n_w2t+1)]
    REGCA1['bus'] = sa.SynGen.get(src='bus', idx=syg_w2t)
    REGCA1['gen'] = sa.SynGen.get(src='gen', idx=syg_w2t)
    REGCA1['Sn'] = sa.SynGen.get(src='Sn', idx=syg_w2t)
    REGCA1['Lvplsw'] = [0] * n_w2t
    REGCA1['gammap'] = sa.SynGen.get(src='gammap', idx=syg_w2t)
    REGCA1['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_w2t)

    # unmapped parameters are skipped and default values are used
    # add to the system
    for row in REGCA1.itertuples(index=False):
        sa.add(model='REGCA1', param_dict={**row._asdict()})

    # turn off the original GENROU
    sa.SynGen.alter(src='u', value=0, idx=syg_w2t)

    # --- GENROU -> WT2G: REECA1 ---

    REECA1 = pd.DataFrame()

    # mapped parameters
    REECA1['u'] = [1] * n_w2t
    REECA1['idx'] = [f'REECA1_{i}' for i in range(1, n_w2t+1)]
    REECA1['reg'] = REGCA1['idx'].copy()

    # unmapped mandatory parameters
    REECA1['PFFLAG'] = [0] * n_w2t
    REECA1['VFLAG'] = [0] * n_w2t
    REECA1['QFLAG'] = [0] * n_w2t
    REECA1['PFLAG'] = [0] * n_w2t
    REECA1['PQFLAG'] = [0] * n_w2t
    REECA1['Vdip'] = [0.8] * n_w2t
    REECA1['Vup'] = [1.2] * n_w2t
    REECA1['Trv'] = [0.02] * n_w2t

    REECA1['dbd1'] = [-0.1] * n_w2t
    REECA1['dbd2'] = [0.1] * n_w2t
    REECA1['Kqv'] = [20] * n_w2t
    REECA1['Thld'] = [-2] * n_w2t
    REECA1['Thld2'] = [1] * n_w2t
    REECA1['Vref1'] = [1] * n_w2t
    REECA1['Imax'] = [10] * n_w2t

    REECA1['Kqp'] = [1] * n_w2t
    REECA1['Kqi'] = [0] * n_w2t
    REECA1['Kvp'] = [1] * n_w2t
    REECA1['Kvi'] = [0] * n_w2t

    # unmapped parameters are skipped and default values are used
    # add to the system
    for row in REECA1.itertuples(index=False):
        sa.add(model='REECA1', param_dict={**row._asdict()})

    # --- GENROU -> WT2G: REPCA1 ---
    REPCA1 = pd.DataFrame()

    # mapped parameters
    REPCA1['u'] = [1] * n_w2t
    REPCA1['idx'] = [f'REPCA1_{i}' for i in range(1, n_w2t+1)]
    REPCA1['ree'] = REECA1['idx']
    # manually assign mandatory param `line`
    ree = REPCA1['ree']
    reg = sa.REECA1.get(src='reg', attr='v', idx=ree)
    bus = sa.REGCA1.get(src='bus', attr='v', idx=reg)
    line1 = sa.Line.find_idx(keys='bus1', values=bus, allow_none=True, default=None)
    line2 = sa.Line.find_idx(keys='bus2', values=bus, allow_none=True, default=None)
    line = []
    for l1, l2 in zip(line1, line2):
        if l1 is not None:
            line.append(l1)
        elif l2 is not None:
            line.append(l2)
        else:
            line.append(None)
    REPCA1['line'] = line
    REPCA1['VCFlag'] = [0] * n_w2t  # with droop
    REPCA1['RefFlag'] = [0] * n_w2t  # Q control
    REPCA1['Fflag'] = [0] * n_w2t  # Frequency control, disabled
    REPCA1['PLflag'] = [0] * n_w2t  # Pline control, disabled

    # unmapped parameters are skipped and default values are used
    # add to the system
    for row in REPCA1.itertuples(index=False):
        sa.add(model='REPCA1', param_dict={**row._asdict()})

    # --- GENROU -> WT2G: WTDS ---
    WTDS = pd.DataFrame()

    # mapped parameters
    WTDS['u'] = [1] * n_w2t
    WTDS['idx'] = [f'WTDS_{i}' for i in range(1, n_w2t+1)]
    WTDS['ree'] = REECA1['idx']
    WTDS['H'] = [3] * n_w2t
    WTDS['D'] = [1] * n_w2t
    WTDS['w0'] = [1] * n_w2t

    for row in WTDS.itertuples(index=False):
        sa.add(model='WTDS', param_dict={**row._asdict()})

### TGOV1N -> [IEEEG1](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Governor%20IEEEG1,%20IEEEG1D%20and%20IEEEG1_GE.htm)

Replace TGOV1N attached to ST type generators with IEEEG1

|   TGOV1N         |   IEEEG1         |   Notes                       |
|------------------|------------------|--------------------------------|
| Tn               | Tn               | Direct mapping                |
| R                | K                | K = 1/R                       |
| T1               | T1               | Direct mapping                |
| Vmax             | Pmax             | Direct mapping                |
| Vmin             | Pmin             | Direct mapping                |
| T2               | T2               | Direct mapping                |
| T3               | T5               | Reheater time constant        |
| Dt               | --               | Discard                       |

Unmapped parameters: T3, Uo, Uc, K1-K8, T4-T7

### TGOV1N -> [GAST](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Governor%20GAST_PTI%20and%20GASTD.htm)

Replace TGOV1N attached to GT type generators with GAST

|   TGOV1N         |   GAST           |   Notes                       |
|------------------|------------------|--------------------------------|
| Tn               | Tn               | Direct mapping                |
| R                | R                | Direct mapping                |
| T1               | T1               | Direct mapping                |
| Vmax             | Vmax             | Direct mapping                |
| Vmin             | Vmin             | Direct mapping                |
| T2               | T2               | Direct mapping                |
| T3               | T3               | Direct mapping                |
| Dt               | Dturb            | Direct mapping                |

Unmapped parameters: Kt, At

### GENROU -> [WT2G](https://www.esig.energy/wiki-main-page/wt2-generic-wind-model/) 

Replace GENROU of W2 type generators to WT2G.

Let's model 

1. REGCA1: This model represents the renewable energy generator/converter dynamics. It is the core model for the generator's electrical behavior.
1. REECA1: This model handles the electrical control aspects, such as voltage regulation and reactive power control. It works in conjunction with REGCA1.
1. REPCA1: This model represents plant-level control, including active power control and coordination of multiple turbines within a wind farm.
1. WTPTA1: This model simulates the pitch control mechanism, which adjusts the blade angle to regulate power output and protect the turbine.
1. WTDTA1: This model represents the drive-train dynamics, including the mechanical coupling between the turbine rotor and the generator.
1. WTARA1: This model simulates the aerodynamic behavior of the wind turbine, including the interaction between wind speed and rotor dynamics.

| GENROU/TGOV1/EXAC1 | Wind Model       | Notes                                        |
|--------------------|------------------|----------------------------------------------|
| H                  | WTPTA1 / WTARA1  | Modeled as equivalent inertia or response    |
| Damping (D)        | --               | Often ignored or included in torque model    |
| TGOV1 T1, T2, T3   | WTPTA1 T1, T2    | Approximate power control lag matching       |
| EXAC1 Ka, Ta       | REECA1 Kpv, Tp   | Voltage regulator gain and filter            |
| Pmax/Pmin          | WTPTA1 Pmax, Pmin| Same as torque/power limits                  |
| Qmax/Qmin          | REECA1 Qmax/Qmin | Reactive power limits                        |
| R (droop)          | WTARA1 Kpg       | Droop gain for active power control          |
| Vref               | REECA1 Vref      | Terminal voltage control setpoint            |
| Pref               | WTPTA1 Pref      | Active power control setpoint                |


#### REGCA1

#### REECA1

#### REPCA1

#### WTDS

RenGovernor

In [9]:
# Export to XLSX for Further Use
andes.io.xlsx.write(sa1, 'ACTIVSg200/1.IL200_dyn_20wt.xlsx',
                    overwrite=True)
andes.io.xlsx.write(sa2, 'ACTIVSg200/3.IL200_dyn_50wt.xlsx',
                    overwrite=True)
andes.io.xlsx.write(sa3, 'ACTIVSg200/5.IL200_dyn_80wt.xlsx',
                    overwrite=True)

True

## Test TDS

In [10]:
for sa in [sa1, sa2, sa3]:

    sa.setup()
    sa.PFlow.run()
    _ = sa.TDS.init()

GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_6  | 0      | 1    
 GENROU_7  | 0.858  | 1    
 GENROU_16 | 0      | 1    
 GENROU_17 | 0      | 1    
 GENROU_20 | 0      | 1    
 GENROU_22 | 0      | 1    
 GENROU_23 | 0      | 1    
 GENROU_24 | 0      | 1    
 GENROU_25 | 0      | 1    
 GENROU_31 | 0      | 1    
 GENROU_37 | 0      | 1    
 GENROU_38 | 0      | 1    
 GENROU_39 | 0      | 1    
 GENROU_40 | 0      | 1    
 GENROU_42 | 0      | 1    
 GENROU_43 | 0      | 1    
 GENROU_48 | 0      | 1    
 GENROU_49 | 0      | 1    


GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_6  | 0      | 1    
 GENROU_7  | 0.858  | 1    
 GENROU_16 | 0      | 1    
 GENROU_17 | 0      | 1    
 GENROU_20 | 0      | 1    
 GENROU_22 | 0      | 1    
 GENROU_23 | 0      | 1    
 GENROU_24 | 0      | 1    
 GENROU_25 | 0      | 1    
 GENROU_26 | 0      | 1    
 GENROU_